<a href="https://colab.research.google.com/github/OmdenaAI/omdena-bangladesh-market-place/blob/task-1/src/tasks/task-1-data-collection/play_store_review_scrapping_for_ecommerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google play scraper is a python library to scrap data from only google play store. We are going to use this to scrap reviews in various ecommerce site apps. Here is the github project link of the library.
https://github.com/JoMingyu/google-play-scraper

In [ ]:
!pip install google-play-scraper

In [ ]:
import pandas as pd
from google_play_scraper import Sort, reviews, app

In [ ]:
# listing the play store app id's of the ecommerce sites

ecommerce_sites_apps = [
    "com.chaldal.poached",
    "com.daraz.android",
    "com.pickaboo.app",
    "com.ajkerdeal.app.android",
    "com.rokomari",
    "com.priyoshop.companion",
    "com.bikroy",
    "xyz.sheba.customersapp",
]

In [ ]:
app_infos = []

for ap in ecommerce_sites_apps:
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)

In [ ]:
app_reviews = []

for ap in ecommerce_sites_apps:
  for score in list(range(6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      result, token = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 200,
        filter_score_with=score
      )
      for r in result:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(result)

In [ ]:
# app_reviews is a list of dictionary which contains various datas of particular review as key value pair
# We are interested in some of the attributes of the reviews
df = pd.DataFrame(app_reviews)

In [ ]:
df_final = df[["appId", "at", "content", "score"]]

In [ ]:
df_final.head()

In [ ]:
changed_name = \
{
    "com.daraz.android":"Daraz", 
    "com.bikroy":"Bikroy",
    "com.ajkerdeal.app.android":"Ajkerdeal",
    "com.chaldal.poached":"Chaldal",
    "com.rokomari":"rokomari",
    "com.pickaboo.app":"Pickaboo",
    "xyz.sheba.customersapp":"Shebaxyz",
    "com.priyoshop.companion":"Priyoshop"
}

In [ ]:
df_final["appId"].replace(changed_name, inplace=True)

In [ ]:
# checking for duplicate reviews those have been extracted more than once

df_final[df_final.duplicate(['content'])]

In [ ]:
# dropping duplicate values

df_final.drop_duplicates(subset="content", keep="first", inplace=True)

In [ ]:
df_final.shape

In [ ]:
# Checking which ecommerce site has how many reviews

df_final.appId.value_counts()

In [ ]:
# checking random reviews

df_final.iloc[7777].content

In [ ]:
# saving the dataframe as a csv file
# df_final.to_csv('play_store_ecommerce_review.csv')